In [ ]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import unicodedata as ud
from sklearn.metrics import precision_recall_fscore_support as prfs
import csv
from itertools import izip
import HTMLParser
from sklearn.metrics import classification_report
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model

In [2]:
def cleanTweet(originalTweet):
    htmlParser = HTMLParser.HTMLParser()

    tweet = originalTweet
    #tweet = htmlParser.unescape(originalTweet);
    #tweet = tweet.decode('windows-1252').encode('ascii', 'ignore')
    #tweet = tweet.decode('windows-1252')
    #tweet = tweet.decode("utf8").encode('ascii', 'ignore')
    #tweet = re.sub(r'[^\x00-\xFF]+', r'', tweet)
    #tweet = re.sub(r'[^\x00-\x7F]+', r'', tweet)
    #tweet = tweet.decode('utf-8').strip()
    #tweet = tweet.decode('unicode_escape').encode('ascii','ignore')
    #tweet = tweet.encode('ascii','ignore')
    tweet = ''.join([i if ord(i) < 128 else ' ' for i in tweet])
    
    # remove URLs in tweet
    tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)

    # remove HTML tags from tweet
    tweet = re.sub('<[^<]+?>', '', tweet)

    # remove strings starting with @ in tweet
    tweet = re.sub(r'(\s)@\w+', r'', tweet)

    # remove exclamations
    tweet = re.sub(r'!', r' ', tweet)

    # separates words joined with capital words.
    # E.g. DisplayIsAweson to Display Is Awesom
    #tweet = " ".join(re.findall('[A-Z][^A-Z]*', tweet));

    # remove extra white spaces
    tweet = re.sub(r'\s+', r' ', tweet)
    
    return tweet

In [1]:
data = pd.read_csv("C:\\Users\\shekh\\Google Drive\\UIC\\UIC Fall 2016 Semester\\Data Mining and Text  Mining\\twitteranalysis\\data\\trainObama_3_columns.csv")

NameError: name 'pd' is not defined

In [4]:
len(data)

5624

In [5]:
rawTweetsSeries = data['Anootated tweet'];
tweetLabels = data['Class'];

In [6]:
rawTweetsList = rawTweetsSeries.tolist()
tweetLabelList = tweetLabels.tolist()

In [7]:
len(rawTweetsList)

5624

In [8]:
# clean all the tweets
i = 0;
cleanedTweetsList = []
for tweet in rawTweetsList:
    #tweet.encode('utf-8').strip()
    #tweet = tweet.decode("utf8").encode('ascii', 'ignore')
    #print (i ,),
    cleanedTweet = cleanTweet(tweet).encode('ascii', 'ignore').strip();
    cleanedTweetsList.append(cleanedTweet);
    
    i += 1

In [9]:
cleanedTweetsList[12]

'@realDonaldTrump I agree we cant afford 4 more years of Obama'

In [10]:
print (i)
print (len(cleanedTweetsList))

5624
5624


In [11]:
#with open('cleanedRomney.csv', 'wb') as f:
#    writer = csv.writer(f)
#    writer.writerows(izip(cleanedTweetsList, tweetLabelList))

In [12]:
trainTweets, testTweets, trainTweetLabels, testTweetLabels = train_test_split(cleanedTweetsList, tweetLabelList, 
                                                                              test_size=0.25, random_state = 40);

In [13]:
len(trainTweetLabels)

4218

In [14]:
# Create feature vectors
vectorizer = TfidfVectorizer(min_df=5,
                             max_df = 0.75,
                             sublinear_tf=True,
                             use_idf=True,
                             stop_words=u'english',
                             analyzer='word',ngram_range=(1,5),lowercase=True)

In [15]:
train_vectors = vectorizer.fit_transform(trainTweets);
test_vectors = vectorizer.transform(testTweets);
total_vectors = vectorizer.fit_transform(cleanedTweetsList);

In [16]:
type(train_vectors)
features = vectorizer.get_feature_names();
indices = np.argsort(vectorizer.idf_)[::-1]
top_n = 20
top_features = [features[i] for i in indices[:top_n]]
print top_features
#print(train_vectors);

#len(train_vectors)

[u'zero', u'indonesia', u'inept civil servant', u'seizure americans bank accounts', u'seizure americans bank', u'seizure americans', u'seizure', u'info', u'inquirer', u'interesting obama', u'interviews', u'investments', u'jesus', u'jewish', u'jews', u'job president', u'jobs overseas', u'johnson', u'jumped', u'seals']


In [17]:
#classifier_rbf = svm.SVC()      default kernel RBF. Very poor results, classify all tweets as -1
#classifier_rbf = svm.SVC(kernel='linear')     decent results, ~50-60 accuracy
#classifier_rbf = svm.LinearSVC()
#knn = KNeighborsClassifier()
#logistic = linear_model.LogisticRegression(C=1e5)
classifier_sgd = linear_model.SGDClassifier()

In [18]:
classifier_sgd.fit(train_vectors, trainTweetLabels)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [19]:
prediction_rbf = classifier_sgd.predict(test_vectors)

In [20]:
predictedLabelList = prediction_rbf.tolist()

In [21]:
classActual = np.array(testTweetLabels)

classPredicted = np.array(predictedLabelList)

In [22]:
classActual
len(classActual)

classPredicted
len(classPredicted)

1406

In [23]:
target_names = ['-1', '0', '1']

In [24]:
prfs(classActual, classPredicted)

(array([ 0.60125261,  0.54545455,  0.59911894]),
 array([ 0.60631579,  0.52332657,  0.62100457]),
 array([ 0.60377358,  0.53416149,  0.60986547]),
 array([475, 493, 438]))

In [25]:
print (len(predictedLabelList))
print (len(testTweetLabels))

1406
1406


In [26]:
correct = 0;
for i in range(len(predictedLabelList)):
    if(predictedLabelList[i] == testTweetLabels[i]):
        correct += 1;

In [27]:
print (correct)

818


In [28]:
predictedLabelList[470]

-1L

In [29]:
import collections
counter=collections.Counter(predictedLabelList)
print(counter)

Counter({-1L: 479, 0L: 473, 1L: 454})


In [30]:
import collections
counter=collections.Counter(testTweetLabels)
print(counter)

Counter({0: 493, -1: 475, 1: 438})


In [31]:
classifier_sgd.score(test_vectors, testTweetLabels)

0.58179231863442393

In [32]:
print(classification_report(testTweetLabels, predictedLabelList, target_names=target_names))

             precision    recall  f1-score   support

         -1       0.60      0.61      0.60       475
          0       0.55      0.52      0.53       493
          1       0.60      0.62      0.61       438

avg / total       0.58      0.58      0.58      1406



In [33]:
cvScores = cross_val_score(classifier_sgd, total_vectors, tweetLabelList, cv=10,scoring='f1_weighted')

In [34]:
print ("Cross validations scores: "),
print (cvScores)

Cross validations scores:  [ 0.55622357  0.48660356  0.57033504  0.57365497  0.55203354  0.59052206
  0.6042658   0.50904247  0.5842544   0.56609024]


In [35]:
print ("Mean: ", cvScores.mean())
print ("Minimum: ", cvScores.min())
print ("Maximum: ", cvScores.max())

('Mean: ', 0.55930256570510983)
('Minimum: ', 0.48660355600289507)
('Maximum: ', 0.60426580474077662)


In [36]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens